https://docs.aws.amazon.com/AmazonS3/latest/user-guide/static-website-hosting.html

https://boto3.amazonaws.com/v1/documentation/api/latest/guide/s3-example-static-web-host.html

In [40]:
import boto3
import json

In [35]:
bucket_name = "grupo5-staticsite"
account_id = "622305974757"

In [18]:

# Create an S3 client
s3 = boto3.client('s3')

response = s3.list_buckets()
bucket_names = [bucket['Name'] for bucket in response['Buckets']]

bucket_exists = False
if bucket_name in bucket_names:
    bucket_exists = True
else:
    bucket_exists = False

if not bucket_exists:
    op_ok = s3.create_bucket(Bucket=bucket_name)    
    print(op_ok)


In [20]:
# Define the website configuration
website_configuration = {
    'ErrorDocument': {'Key': 'error.html'},
    'IndexDocument': {'Suffix': 'index.html'},
}

# Set the website configuration
s3 = boto3.client('s3')
s3.put_bucket_website(Bucket=bucket_name,
                      WebsiteConfiguration=website_configuration)

{'ResponseMetadata': {'RequestId': '4FAE12835F5E2E23',
  'HostId': 'rFN+GB0+lWPYTnlTbphB7fhR0/P3uj8ngd0SZOLgajnW4W9lE54nkTyelwQeGGYmhLnfd9hCSA8=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'rFN+GB0+lWPYTnlTbphB7fhR0/P3uj8ngd0SZOLgajnW4W9lE54nkTyelwQeGGYmhLnfd9hCSA8=',
   'x-amz-request-id': '4FAE12835F5E2E23',
   'date': 'Fri, 13 Dec 2019 15:54:22 GMT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [21]:
result = s3.get_bucket_website(Bucket=bucket_name)

In [ ]:
# Delete the website configuration
# s3 = boto3.client('s3')
# s3.delete_bucket_website(Bucket='BUCKET_NAME')


disable block public access for the bucket:

In [36]:
client = boto3.client('s3control')
client.delete_public_access_block(AccountId= account_id)

{'ResponseMetadata': {'RequestId': 'FD7F120A2E0BAFEA',
  'HostId': 'M75FID38uS9cxvOg1A6YJNl+9HRjvU0BE7pgipN9JM+CvimhAlg/GaH1/bSkXl4vdQAs7E94pKc=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'M75FID38uS9cxvOg1A6YJNl+9HRjvU0BE7pgipN9JM+CvimhAlg/GaH1/bSkXl4vdQAs7E94pKc=',
   'x-amz-request-id': 'FD7F120A2E0BAFEA',
   'date': 'Fri, 13 Dec 2019 16:14:46 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

Add a bucket policy to the website bucket that grants everyone access to the objects in the bucket.

In [46]:
# Create a bucket policy
bucket_name = bucket_name

bucket_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "PublicReadGetObject",
            "Effect": "Allow",
            "Principal": "*",
            "Action": [
                "s3:GetObject"
            ],
            "Resource": [
                f'arn:aws:s3:::{bucket_name}/*', 
                f'arn:aws:s3:::{bucket_name}/*/*'
            ]
        }
    ]
}

#{
#    'Version': '2012-10-17',
#    'Statement': [{
#        'Sid': 'AddPerm',
#        'Effect': 'Allow',
#        'Principal': '*',
#        'Action': ['s3:GetObject'],
#        'Resource': f'arn:aws:s3:::{bucket_name}/*'
#    }]
#}

# Convert the policy from JSON dict to string
bucket_policy = json.dumps(bucket_policy)

s3.put_bucket_policy(Bucket=bucket_name, Policy=bucket_policy)

{'ResponseMetadata': {'RequestId': 'B1829988A0FEA8B8',
  'HostId': '/ARpLKYCTMbqI7u08NI7QZmIJ2+JSrtFEC/Kd6kiA2o+aQfyCYxhk11uD+NwxC0+twfkg33WYq0=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': '/ARpLKYCTMbqI7u08NI7QZmIJ2+JSrtFEC/Kd6kiA2o+aQfyCYxhk11uD+NwxC0+twfkg33WYq0=',
   'x-amz-request-id': 'B1829988A0FEA8B8',
   'date': 'Fri, 13 Dec 2019 17:26:48 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [42]:
# Retrieve the policy of the specified bucket
result = s3.get_bucket_policy(Bucket=bucket_name)
print(result['Policy'])

{"Version":"2012-10-17","Statement":[{"Sid":"AddPerm","Effect":"Allow","Principal":"*","Action":"s3:GetObject","Resource":"arn:aws:s3:::grupo5-staticsite/*"}]}
